In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv("../data/tweets_labeled.csv", encoding="utf-8", delimiter="\t")

In [3]:
tweets.head()

,id,text,sentiment
0,1,@caprichOreality Fica assim não miga &lt;3 Tud...,1.0
1,2,Parti me todo a descer a avenida de Gaia com o...,1.0
2,3,Amanhã é dia de dar um trato na palestra para ...,1.0
3,4,@thankovsky @patorebaichado eu também tenho :)...,1.0
4,5,ok. Sim. Aham. Tá. De boa. Vai lá. :) https://...,1.0


In [4]:
tweets.shape

(57483, 3)

### Defining some constants

In [5]:
COL_TOKENIZED_TEXT = 'tokenized_text'
COL_TEXT = 'text'
POSITIVE = 'positive'
NEGATIVE = 'negative'
COL_PREDICT = 'predict'

## Cleaning

In [6]:
import re
import nltk

def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def remove_links(text):
    return re.sub(r"http\S+", "", text)

def remove_mentions(text):
    return re.sub(r"@\w+", "", text)

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text) # remove special chars
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
    text_with_no_special_chars = re.sub("\s+", " ", text) #remove all duplicated spaces
    return text_with_no_special_chars

def stemming(text):
    stemmer = nltk.stem.RSLPStemmer()
    words = []
    for word in text.split():
        words.append(stemmer.stem(word))
    return (" ".join(words))

def standardize_text(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_mentions(text)
    text = remove_stopwords(text)
    text = remove_special_chars(text)
    return text

Removendo stopwords, alguns caracteres especiais, links e mentions e alterando para lowercase.

In [7]:
tweets.text = tweets.text.apply(standardize_text)

Removendo tweets com textos iguais (ex: retweets)

In [8]:
tweets = tweets.drop_duplicates(COL_TEXT)

In [9]:
tweets.shape

(52760, 3)

In [10]:
tweets.head()

,id,text,sentiment
0,1,fica assim miga lt tudo arranja deus quiser,1.0
1,2,parti todo descer avenida gaia skate,1.0
2,3,amanhã é dia dar trato palestra thedevconf aju...,1.0
3,4,posso sentar vocês,1.0
4,5,ok sim aham tá boa vai lá,1.0


#### Toques finais

In [11]:
def get_sentiment_category(sentiment_float):
    if sentiment_float == 1.0:
        return POSITIVE
    else:
        return NEGATIVE
    
tweets.sentiment = tweets.sentiment.apply(get_sentiment_category)

## Creating Model

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

Using [scikit lib](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)

In [13]:
tweets_texts = tweets.text.values
classes = tweets.sentiment.values

In [14]:
vectorizer = CountVectorizer(analyzer="word")
#vectorizer = CountVectorizer(ngram_range = (1, 2))
#vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000)
freq_tweets = vectorizer.fit_transform(tweets_texts)
model = MultinomialNB(alpha=1)
model.fit(freq_tweets, classes)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

## Testing Model

#### Algumas verificações de sanidade..

In [15]:
tests = [
    'Esse governo está no início, vamos ver o que vai dar',
    'Estou muito feliz com o governo de Minas esse ano',
    'O estado de Minas Gerais decretou calamidade financeira!!!',
    'A segurança desse país está deixando a desejar',
    'O governador de Minas é do PT',
    'Infeliz',
    'Menino to muito doente',
    'Lula 2018',
    'Tô indo para Universidade']

tests = list(map(standardize_text, tests))
freq_testes = vectorizer.transform(tests)

In [16]:
model.predict(freq_testes)

array(['positive', 'positive', 'negative', 'negative', 'positive',
       'negative', 'negative', 'positive', 'negative'], dtype='<U8')

#### Testando com dados reais do Twitter (tweets não usados para treino)

In [17]:
tweets_test = pd.read_csv("../data/tweets_test.csv", encoding="utf-8", delimiter="\t")

In [18]:
tweets_test.head()

,text
0,RT @prosapolitica: Quando começar o Horário El...
1,RT @prosapolitica: Quando começar o Horário El...
2,Convenções definem Dalben e Henrique do Paraís...
3,RT @prosapolitica: Enquete de monitoramento te...
4,RT @prosapolitica: Enquete de monitoramento te...


In [19]:
tweets_test['rawtext'] = tweets_test.text

Retirando os tweets que são retweets (começam com 'RT'), pois no geral são tweets incompletos com palavras quebradas ao meio.

In [20]:
tweets_test = tweets_test.loc[~tweets_test[COL_TEXT].str.contains("RT @")]

In [21]:
tweets_test.shape

(3681, 2)

In [22]:
tweets_test.text = tweets_test.text.apply(standardize_text)

In [23]:
tweets_test.head()

,text,rawtext
2,convenções definem dalben henrique paraíso can...,Convenções definem Dalben e Henrique do Paraís...
6,videogame é crime coibir máximo possível apre...,"""videogame é um crime. Você tem que coibir o m..."
7,eleitor pode usar internet pesquisar sobre can...,O eleitor pode usar a internet para pesquisar ...
20,rede ainda definiu sebastião carlos candidato ...,"Rede ainda definiu Sebastião Carlos, como cand..."
22,vamos informar mais debates incríveis sobre po...,Vamos nos informar mais! Debates incríveis sob...


In [24]:
tweets_test = tweets_test.drop_duplicates(COL_TEXT)

In [25]:
tweets_test.shape

(3470, 2)

In [26]:
freq_testes = vectorizer.transform(tweets_test.text)

In [27]:
tweets_test[COL_PREDICT] = model.predict(freq_testes)

In [28]:
positive_classified = tweets_test[ tweets_test[COL_PREDICT] == POSITIVE]
negative_classified = tweets_test[ tweets_test[COL_PREDICT] == NEGATIVE]

print("Examples of tweets predicted as positive...\n")

for tweet in positive_classified.rawtext.head(10):
    tweet = re.sub("\s+", " ", tweet)
    print(tweet)
    print('---')
    
print("\n\nExamples of tweets predicted as negative...\n")

for tweet in negative_classified.rawtext.head(10):
    tweet = re.sub("\s+", " ", tweet)
    print(tweet)
    print('---')    

Examples of tweets predicted as positive...

Convenções definem Dalben e Henrique do Paraíso candidatos #SegundaDetremuraSDV #Notícias #Brasil #BRNews #Política… https://t.co/9ZA52jbsgO
---
"videogame é um crime. Você tem que coibir o máximo possível, não aprende nada". Cuidado com seu voto nas eleições… https://t.co/AQiHZplAUJ
---
O eleitor pode usar a internet para pesquisar sobre os candidatos em sites como o da Câmara, do Senado, das assembl… https://t.co/RKdNuhk3vm
---
Rede ainda definiu Sebastião Carlos, como candidato ao Senado. #VGNotícias #PolíticaMT #Eleições2018 #MT #REDE… https://t.co/qW20PdUWLB
---
Vamos nos informar mais! Debates incríveis sobre política acontecem na #CBN. Fiquem ligados! #Eleições2018 #NoArNaCBN
---
Daniel Finizola realiza visita ao lado de Teresa Leitão a Camocim de São Félix. Leia no https://t.co/3g8PetzyeA… https://t.co/Qj8vJpGamR
---
Cacoete autoritário https://t.co/gT9OTpZd0X #PT #Eleicoes2018 #Lula #Dilma #Democracia #LavaJato
---
Nacional determin

Os resultados não são muito bons porque os dados usados para treino rotularam de modo muito simplista o sentimento do tweet (se o tweet possui um ':)' ele é positivo; se possui um ':(', é negativo).

## Evaluating Model

A função **cross_val_predict** (validação cruzada do modelo) divide os dados do modelo em 10 partes, treina o modelo com nove e testa com uma.

In [29]:
results = cross_val_predict(model, freq_tweets, classes, cv = 10)

In [30]:
accuracy = metrics.accuracy_score(classes, results)
print("A acurácia do modelo é de {0:.2f}".format(accuracy))

A acurácia do modelo é de 0.74


In [31]:
sentiments = [POSITIVE, NEGATIVE]
print(metrics.classification_report(classes, results, sentiments))

             precision    recall  f1-score   support

   positive       0.77      0.70      0.73     27104
   negative       0.71      0.78      0.74     25656

avg / total       0.74      0.74      0.74     52760



# Notes

- Sem stemming ficou melhor
- Melhor modelo: o com bigrams
- https://github.com/minerandodados/mdrepo/blob/master/Notebook_AnaliseSentimentosGovernoMinas_MinerandoDados.ipynb
- https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese